In [1]:
# %load requirements.txt

import findspark
findspark.init()

import pyspark
print(findspark.find())

from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("Spark Practice")

sc = SparkContext(conf=conf)

from pyspark.sql import SparkSession
spark = SparkSession(sc)


/opt/spark


# 1. 테이블 생성하기

In [2]:
df = spark.read.json("./people.json")

In [3]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
# DataFrame으로 뷰를 생성 
df.createOrReplaceTempView("people")

# 2. 기본 SQL 처리하기

In [9]:
table = "people"

In [10]:
fstr = f"SELECT * FROM {table}"

In [11]:
fstr

'SELECT * FROM people'

In [12]:
spark.sql(fstr)

DataFrame[age: bigint, name: string]

In [13]:
spark.sql(fstr).explain()

== Physical Plan ==
*(1) FileScan json [age#6L,name#7] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/dahlmoon/Documents/GitHub/pyspark/people.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<age:bigint,name:string>


In [14]:
spark.sql(fstr).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [15]:
spark.sql(f"SELECT * FROM {table}").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [16]:
spark.sql("SELECT * FROM people WHERE age > 20").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## 그룹화하기 

In [17]:
spark.sql("SELECT age, count(1) FROM people GROUP BY age").show()

+----+--------+
| age|count(1)|
+----+--------+
|  19|       1|
|null|       1|
|  30|       1|
+----+--------+



# 3. 조인하기

## sql로 조인하기

In [18]:
spark.sql("""SELECT a.name, b.age 
       FROM people a, people b
       WHERE a.name = b.name""").show()

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+



## 데이터프레임을 사용한 조인

In [20]:
peopleA = spark.sql("select name, age as a_age from people")
peopleB = spark.sql("select name, age as b_age from people")

In [21]:
peopleA

DataFrame[name: string, a_age: bigint]

In [22]:
peopleB

DataFrame[name: string, b_age: bigint]

In [23]:
peopleA.join(peopleB, "name").select("*").show()

+-------+-----+-----+
|   name|a_age|b_age|
+-------+-----+-----+
|Michael| null| null|
|   Andy|   30|   30|
| Justin|   19|   19|
+-------+-----+-----+



In [24]:
peopleA.join(peopleB, "name").select("a_age").show()

+-----+
|a_age|
+-----+
| null|
|   30|
|   19|
+-----+



# 4. 파일을 읽고 테이블 만들기

## SparkFiles 클래스 사용해서 데이터 가져오기

In [25]:
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
from pyspark import SparkFiles
sc.addFile(url)

In [26]:
df = spark.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema= True)

In [27]:
df['x']

Column<b'x'>

In [28]:
df.show(1)

+---+---+---------+------+---------+---------------+--------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  x|age|workclass|fnlwgt|education|educational-num|marital-status|       occupation|relationship| race|gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+---------+---------------+--------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|  1| 25|  Private|226802|     11th|              7| Never-married|Machine-op-inspct|   Own-child|Black|  Male|           0|           0|            40| United-States| <=50K|
+---+---+---------+------+---------+---------------+--------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
only showing top 1 row



In [29]:
df.count()

48842

In [30]:
# DataFrame으로 뷰를 생성 
df.createOrReplaceTempView("adult")

In [31]:
df.printSchema()

root
 |-- x: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [32]:
df.columns

['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [33]:
df.take(1)

[Row(x=1, age=25, workclass='Private', fnlwgt=226802, education='11th', educational-num=7, marital-status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital-gain=0, capital-loss=0, hours-per-week=40, native-country='United-States', income='<=50K')]

In [34]:
df.head()

Row(x=1, age=25, workclass='Private', fnlwgt=226802, education='11th', educational-num=7, marital-status='Never-married', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capital-gain=0, capital-loss=0, hours-per-week=40, native-country='United-States', income='<=50K')

## 스키마 자료형  추론하지 않으면 문자열로 처리

In [35]:
df_string = spark.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema=  False)
df_string.printSchema()

root
 |-- x: string (nullable = true)
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: string (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: string (nullable = true)
 |-- hours-per-week: string (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



## 자료형을 변경한다

In [46]:
# Import all from `sql.types`
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 
# List of continuous features
CONTI_FEATURES  = ['age', 'fnlwgt','capital-gain', 'educational-num', 'capital-loss', 'hours-per-week']
# Convert the type
df_string = convertColumn(df_string, CONTI_FEATURES, FloatType())
# Check the dataset
df_string.printSchema()

root
 |-- x: string (nullable = true)
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- educational-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [36]:
df.select('age','fnlwgt').show(5)

+---+------+
|age|fnlwgt|
+---+------+
| 25|226802|
| 38| 89814|
| 28|336951|
| 44|160323|
| 18|103497|
+---+------+
only showing top 5 rows



In [39]:
df.groupBy("education").count().sort("count",ascending=True).show()

+------------+-----+
|   education|count|
+------------+-----+
|   Preschool|   83|
|     1st-4th|  247|
|     5th-6th|  509|
|   Doctorate|  594|
|        12th|  657|
|         9th|  756|
| Prof-school|  834|
|     7th-8th|  955|
|        10th| 1389|
|  Assoc-acdm| 1601|
|        11th| 1812|
|   Assoc-voc| 2061|
|     Masters| 2657|
|   Bachelors| 8025|
|Some-college|10878|
|     HS-grad|15784|
+------------+-----+



In [40]:
df.groupBy("education").agg({'capital-gain': 'mean'}).show()

+------------+------------------+
|   education| avg(capital-gain)|
+------------+------------------+
|        10th| 323.0496760259179|
|     Masters|2583.6059465562666|
|     5th-6th|360.36542239685656|
|  Assoc-acdm|  636.951905059338|
|   Assoc-voc| 778.6021348859776|
|     7th-8th| 242.6261780104712|
|         9th|313.39814814814815|
|     HS-grad| 573.3141789153573|
|   Bachelors| 1762.564984423676|
|        11th|203.73951434878586|
|     1st-4th| 123.5910931174089|
|   Preschool|             732.0|
|        12th| 208.5799086757991|
|   Doctorate|  5727.76936026936|
|Some-college| 559.9615738187167|
| Prof-school| 10586.46762589928|
+------------+------------------+



In [49]:
df.describe().show()

+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|summary|                 x|               age|  workclass|            fnlwgt|   education|   educational-num|marital-status|      occupation|relationship|              race|gender|      capital-gain|      capital-loss|    hours-per-week|native-country|income|
+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|  count|             48842|             48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|             4884

In [50]:
df.summary().show()

+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|summary|                 x|               age|  workclass|            fnlwgt|   education|   educational-num|marital-status|      occupation|relationship|              race|gender|      capital-gain|      capital-loss|    hours-per-week|native-country|income|
+-------+------------------+------------------+-----------+------------------+------------+------------------+--------------+----------------+------------+------------------+------+------------------+------------------+------------------+--------------+------+
|  count|             48842|             48842|      48842|             48842|       48842|             48842|         48842|           48842|       48842|             48842| 48842|             48842|             4884

In [51]:
df.describe('capital-gain').show()

+-------+------------------+
|summary|      capital-gain|
+-------+------------------+
|  count|             48842|
|   mean|1079.0676262233324|
| stddev| 7452.019057655406|
|    min|                 0|
|    max|             99999|
+-------+------------------+



In [60]:
df.crosstab('age', 'race').sort("age_race").show()

+--------+------------------+------------------+-----+-----+-----+
|age_race|Amer-Indian-Eskimo|Asian-Pac-Islander|Black|Other|White|
+--------+------------------+------------------+-----+-----+-----+
|      17|                 6|                 2|   45|    8|  534|
|      18|                 8|                18|   72|   13|  751|
|      19|                10|                24|   87|    5|  927|
|      20|                 8|                25|   93|   16|  971|
|      21|                 9|                28|  101|   15|  943|
|      22|                16|                43|  115|   10|  994|
|      23|                13|                48|  132|   15| 1121|
|      24|                 9|                49|  122|   14| 1012|
|      25|                 9|                38|  123|   15| 1010|
|      26|                 7|                42|  127|   18|  959|
|      27|                16|                52|  140|   18| 1006|
|      28|                20|                35|  147|   18| 1

In [61]:
df.drop('educational-num').columns


['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [62]:
df.columns

['x',
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [63]:
df.filter(df.age > 40).count()	

20211

In [71]:
df.select('marital-status').distinct().show()

+--------------------+
|      marital-status|
+--------------------+
|           Separated|
|       Never-married|
|Married-spouse-ab...|
|            Divorced|
|             Widowed|
|   Married-AF-spouse|
|  Married-civ-spouse|
+--------------------+



In [66]:
df.groupby('marital-status').agg({'capital-gain': 'mean'}).show()

+--------------------+------------------+
|      marital-status| avg(capital-gain)|
+--------------------+------------------+
|           Separated| 581.8424836601307|
|       Never-married|  384.382639449029|
|Married-spouse-ab...| 629.0047770700637|
|            Divorced| 793.6755615860094|
|             Widowed| 603.6442687747035|
|   Married-AF-spouse|2971.6216216216217|
|  Married-civ-spouse|1739.7006121810625|
+--------------------+------------------+

